## Volatility is one of the most important terms.

### Import

In [1]:
import os
import sys
import time
import glob
from pathlib import Path

import pandas as pd
import numpy as np

# Parallel processing
from joblib import Parallel
from joblib import delayed

# Preprocess
from sklearn import preprocessing
from sklearn import model_selection

# Evaluation

from sklearn.metrics import r2_score
# Visullize
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# Modeling
import lightgbm as lgb
from sklearn.model_selection import KFold
# Others
import warnings
warnings.simplefilter("ignore")

### Setting

In [2]:
# data_path = Path('C:/Users/markr/Desktop/kaggle/optiver-realized-volatility-prediction')
data_path = Path('C:/Users/markluo/Desktop/Creation Mark/1.學習資料/11.kaggle/Optiver Realized Volatility Prediction')
pd.options.display.max_columns = 30
pd.options.display.max_rows = 9
list_order_book_file_train = glob.glob('book_train.parquet/stock_id=*')
list_order_book_file_test = glob.glob('book_test.parquet/stock_id=*')

### Function

In [5]:
# return
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()
# realized volatility
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
# WAP 
def BidAskSpread(df):
    return ((df['ask_price1']/df['bid_price1'])-1)
def Wap1(df):
    return (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
def Wap2(df):
    return (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
# Gap price
def BidAsk_price_gap(df):
    return (df['ask_price1']-df['bid_price1'])
def BidAsk_price_center(df):
    return ((df['ask_price1']+df['bid_price1'])/2)
# Gap size
def BidAsk_size_gap(df):
    return (abs(df['ask_size1']-df['bid_size1']))
def BidAsk_size_center(df):
    return ((df['ask_size1']+df['bid_size1'])/2)
# Weighted  
def BidAsk_Weighted_center(df):
    gap= df['ask_price1']-df['bid_price1']
    center= ((df['ask_price1']+df['bid_price1'])/2)
    add_weight=abs((df['ask_price1']-center)*df['ask_size1'])
    mins_weight=abs((df['bid_price1']-center)*df['bid_size1'])
    weight_center=center+gap*(add_weight-mins_weight)/(add_weight+mins_weight)
    return (weight_center)
def Bid_spread(df):
    return (df['bid_price1'] - df['bid_price2'])
def Ask_spread(df):
    return (abs(df['ask_price1'] - df['ask_price2']))
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def bid_price1_pre(df):
    return df['bid_price1'].shift(1)
def ask_price1_pre(df):
    return df['ask_price1'].shift(1)
def bid_size1_pre(df):
    return df['bid_size1'].shift(1)
def ask_size1_pre(df):
    return df['ask_size1'].shift(1)


    
def Book_make_feature(stock_id : int, data_type = 'train'):
    # read data
    df = pd.read_parquet(data_path / f'book_{data_type}.parquet/stock_id={stock_id}/')
    df['stock_id'] = stock_id
    # feature
    df['BidAskSpread'] = BidAskSpread(df)
    df['Wap1'] = Wap1(df)
    df['Wap2'] = Wap2(df)
    df['BidAsk_price_gap'] = BidAsk_price_gap(df)
    df['BidAsk_price_center'] = BidAsk_price_center(df)
    df['BidAsk_size_gap'] = BidAsk_size_gap(df)
    df['BidAsk_size_center'] = BidAsk_size_center(df)
    df['BidAsk_Weighted_center'] = BidAsk_Weighted_center(df)
    df['Bid_spread'] = Bid_spread(df)
    df['Ask_spread'] = Ask_spread(df)
    df['bid_price1_pre'] =bid_price1_pre(df)
    df['ask_price1_pre'] =ask_price1_pre(df)
    df['bid_size1_pre'] =bid_size1_pre(df)
    df['ask_size1_pre'] = ask_size1_pre(df)

    # log return calculation
    df['log_return1'] = df.groupby(['time_id'])['Wap1'].apply(log_return).fillna(0)
    df['log_return2'] = df.groupby(['time_id'])['Wap2'].apply(log_return).fillna(0)    
    df=df.reset_index()
    return df

def Book_make_derivative_feature(stock_id : int, data_type = 'train'):
    df=Book_make_feature(stock_id=stock_id, data_type = data_type)
    # Log_return calculation each stock_id and time_id
    df_return = pd.DataFrame(df.groupby(['stock_id','time_id'])\
                                             [['log_return1','log_return2']].agg(realized_volatility)).reset_index()
    # max
#     df_max=df.groupby(['stock_id','time_id'])\
#                         [['bid_price1','ask_price1','bid_size1','ask_size1','Wap1','BidAsk_price_gap','Bid_spread','Ask_spread']].agg(max).reset_index()
#     df_max.columns = ['stock_id', 'time_id',\
#                       'bid_price1_max','ask_price1_max','bid_size1_max','ask_size1_max','Wap1_max','BidAsk_price_gap_max','Bid_spread_max','Ask_spread_max']
    # min
#     df_min=df.groupby(['stock_id','time_id'])\
#                         [['bid_price1','ask_price1','bid_size1','ask_size1','Wap1','BidAsk_price_gap','Bid_spread','Ask_spread']].agg(min).reset_index()
#     df_min.columns = ['stock_id', 'time_id',\
#                       'bid_price1_min','ask_price1_min','bid_size1_min','ask_size1_min','Wap1_min','BidAsk_price_gap_min','Bid_spread_min','Ask_spread_min']
    # median
    df_median=df.groupby(['stock_id','time_id'])\
                        [['Wap1','Wap2','BidAsk_price_gap','Bid_spread','Ask_spread','BidAsk_Weighted_center']].median().reset_index()
    df_median.columns = ['stock_id', 'time_id',\
                      'Wap1_median','Wap2_median','BidAsk_price_gap_median','Bid_spread_median','Ask_spread_median',\
                         'BidAsk_Weighted_center_center']
    # mean
    df_mean=df.groupby(['stock_id','time_id'])\
                        [['Wap1','Wap2','BidAsk_price_gap','Bid_spread','Ask_spread']].mean().reset_index()
    df_mean.columns = ['stock_id', 'time_id',\
                      'Wap1_mean','Wap2_mean','BidAsk_price_gap_mean','Bid_spread_mean','Ask_spread_mean']
    # std
    df_std=df.groupby(['stock_id','time_id'])\
                        [['bid_price1','ask_price1','bid_size1','ask_size1','Wap1','Wap2','BidAsk_price_gap','Bid_spread','Ask_spread']].std().reset_index()
    df_std.columns = ['stock_id', 'time_id',\
                      'bid_price1_std','ask_price1_std','bid_size1_std','ask_size1_std','Wap1_std','Wap2_std','BidAsk_price_gap_std','Bid_spread_min','Ask_spread_std']
    # dataset
    df = df[['stock_id', 'time_id','bid_price1_pre','ask_price1_pre','bid_size1_pre','ask_size1_pre']]
    df = df.merge(df_return, on=['stock_id', 'time_id'], how='left')
#     df = df.merge(df_max, on=['stock_id', 'time_id'], how='left')
#     df = df.merge(df_min, on=['stock_id', 'time_id'], how='left')
    df = df.merge(df_median, on=['stock_id', 'time_id'], how='left')
    df = df.merge(df_mean, on=['stock_id', 'time_id'], how='left')
    df = df.merge(df_std, on=['stock_id', 'time_id'], how='left')

#     df=df.drop('index',axis=1)
#     .reset_index()
    return df
def trade_make_feature(stock_id : int, data_type = 'train'):
    df = pd.read_parquet(data_path / f'trade_{data_type}.parquet/stock_id={stock_id}/')
    df['stock_id'] = stock_id
    df_copy = df.copy()
    df_copy['about_price']=round(df_copy['price'],2)
    df_copy=df_copy.groupby(['about_price']).sum().reset_index()
    max_price_range=df_copy[df_copy['size']==df_copy['size'].max()]['about_price']
    max_price_size=df_copy[df_copy['size']==df_copy['size'].max()]['size']
    df_copy['max_price_range']=float(max_price_range)
    df_copy['max_price_size']=float(max_price_size)
    df = df.sort_values(by=['time_id', 'seconds_in_bucket']).reset_index(drop=True)
    df['trade_log_return1'] = df.groupby(by = ['time_id'])['price'].apply(log_return).fillna(0)
    df = pd.DataFrame(df.groupby(['stock_id','time_id'])[['trade_log_return1']].agg(realized_volatility).reset_index())
    df = pd.merge(df, df_copy, on = ['stock_id', 'time_id'], how = 'left').fillna(0)
    return df
def merge_book_trade(stock_id : int, data_type = 'train'):
    book = Book_make_derivative_feature(stock_id, data_type)
    trade = trade_make_feature(stock_id, data_type)
    dataset = book.merge(trade, on=['stock_id', 'time_id'], how='left').fillna(0)   
    return dataset
def per_stock(list_file,data_type='train'):
    data = pd.DataFrame()
    i=0
    for file in list_file:
        name=file.split('=', 1 )[1]
        name=int(name)
        data = pd.concat([data,merge_book_trade(name,data_type)])
        i=i+1
        print(round(i/len(list_file)*100,2),'%')
    return data

def add_y(list_file,data_type='train'):
    df=per_stock(list_file,data_type=data_type)
    y=pd.read_csv(f'{data_type}.csv')
    df = pd.merge(y, df, on = ['stock_id', 'time_id'], how = 'left').fillna(0)
    df['row_id'] = df['stock_id'].astype(str) + '-' + df['time_id'].astype(str)
    return df

# Define loss function for   training
def feval_rmspe(preds, train_data):
    labels = train_data.get_label()
    return 'RMSPE', rmspe(y_true = labels, y_pred = preds), False

In [4]:
pd.read_parquet(data_path / f'trade_test.parquet/stock_id=0/')

,time_id,seconds_in_bucket,price,size,order_count
0,4,7,1.000344,1,1
1,4,24,1.000049,100,7
2,4,27,1.000059,100,3


In [153]:
df = pd.read_parquet(data_path / f'trade_train.parquet/stock_id=0/')
df['stock_id'] = 0
df = df.sort_values(by=['time_id', 'seconds_in_bucket']).reset_index(drop=True)
df['trade_log_return1'] = df.groupby(by = ['time_id'])['price'].apply(log_return).fillna(0)
df = pd.DataFrame(df.groupby(['stock_id','time_id'])[['trade_log_return1']].agg(realized_volatility).reset_index())
df = pd.merge(df, df_return, on = ['stock_id', 'time_id'], how = 'left').fillna(0)

In [154]:
df_return

,stock_id,time_id,trade_log_return1
0,0,5,0.002006
1,0,11,0.000901
2,0,16,0.001961
3,0,31,0.001561
...,...,...,...
3826,0,32753,0.001411
3827,0,32758,0.001521
3828,0,32763,0.001794
3829,0,32767,0.001197


In [145]:
df_copy = df.copy()
df_copy['about_price']=round(df_copy['price'],2)
df_copy=df_copy.groupby(['about_price']).sum().reset_index()
max_price_range=df_copy[df_copy['size']==df_copy['size'].max()]['about_price']
max_price_size=df_copy[df_copy['size']==df_copy['size'].max()]['size']
df_copy['max_price_range']=float(max_price_range)
df_copy['max_price_size']=float(max_price_size)
df_copy

,about_price,time_id,seconds_in_bucket,price,size,order_count,stock_id,max_price_range,max_price_size
0,0.94,69760.0,2035.0,3.776688,1140,31.0,0,1.0,11014712.0
1,0.95,186590.0,10431.0,21.885500,1878,57.0,0,1.0,11014712.0
2,0.96,833887.0,12276.0,34.609039,11128,250.0,0,1.0,11014712.0
3,0.97,1320951.0,18221.0,70.935738,12906,320.0,0,1.0,11014712.0
...,...,...,...,...,...,...,...,...,...
8,1.02,7064605.0,136019.0,448.037689,62341,1733.0,0,1.0,11014712.0
9,1.03,815833.0,15538.0,39.088749,6629,161.0,0,1.0,11014712.0
10,1.04,1017427.0,9921.0,37.428417,5290,113.0,0,1.0,11014712.0
11,1.05,60256.0,690.0,2.090990,630,8.0,0,1.0,11014712.0


In [ ]:
df = pd.read_parquet(data_path / f'trade_train.parquet/stock_id=0/')
df['stock_id'] = 0
df_copy = df.copy()
df_copy['about_price']=round(df_copy['price'],2)
df_copy=df_copy.groupby(['about_price']).sum().reset_index()
max_price_range=df_copy[df_copy['size']==df_copy['size'].max()]['about_price']
max_price_size=df_copy[df_copy['size']==df_copy['size'].max()]['size']
df_copy['max_price_range']=float(max_price_range)
df_copy['max_price_size']=float(max_price_size)
df = df.sort_values(by=['time_id', 'seconds_in_bucket']).reset_index(drop=True)
df['trade_log_return1'] = df.groupby(by = ['time_id'])['price'].apply(log_return).fillna(0)
df_return = pd.DataFrame(df.groupby(['stock_id','time_id'])[['trade_log_return1']].agg(realized_volatility).reset_index())
df = pd.merge(df, df_return, on = ['stock_id', 'time_id'], how = 'left').fillna(0)

In [157]:
trade_make_feature(0)

,stock_id,time_id,trade_log_return1,about_price,seconds_in_bucket,price,size,order_count,max_price_range,max_price_size
0,0,5,0.002006,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,11,0.000901,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,16,0.001961,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,31,0.001561,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
3826,0,32753,0.001411,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3827,0,32758,0.001521,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3828,0,32763,0.001794,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3829,0,32767,0.001197,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [130]:
trade_train_stock_id_0=pd.read_parquet(data_path / f'trade_train.parquet/stock_id=0/')

In [ ]:
trade_train_stock_id_0['about_price']=round(trade_train_stock_id_0['price'],2)
trade_train_stock_id_0=trade_train_stock_id_0.groupby(['about_price']).sum().reset_index()
max_price_range=trade_train_stock_id_0[trade_train_stock_id_0['size']==trade_train_stock_id_0['size'].max()]['about_price']
max_price_size=trade_train_stock_id_0[trade_train_stock_id_0['size']==trade_train_stock_id_0['size'].max()]['size']
trade_train_stock_id_0['max_price_range']=float(max_price_range)
trade_train_stock_id_0['max_price_size']=float(max_price_size)

In [132]:
trade_train_stock_id_0['about_price']=round(trade_train_stock_id_0['price'],2)

In [133]:
trade_train_stock_id_0=trade_train_stock_id_0.groupby(['about_price']).sum().reset_index()
# trade_train_stock_id_0=trade_train_stock_id_0.groupby(['time_id','about_price']).sum().reset_index()

In [108]:
# max price range
max_price_range=trade_train_stock_id_0[trade_train_stock_id_0['size']==trade_train_stock_id_0['size'].max()]['about_price']

In [109]:
# max price size
max_price_size=trade_train_stock_id_0[trade_train_stock_id_0['size']==trade_train_stock_id_0['size'].max()]['size']

In [111]:
trade_train_stock_id_0['max_price_range']=float(max_price_range)
trade_train_stock_id_0['max_price_size']=float(max_price_size)

In [7]:
test=add_y(list_order_book_file_test,data_type='test').reset_index()
print(f'Test shape: {test.shape}')

100.0 %
Test shape: (5, 38)


In [8]:
test

,index,stock_id,time_id,row_id,bid_price1_pre,ask_price1_pre,bid_size1_pre,ask_size1_pre,log_return1,log_return2,Wap1_median,Wap2_median,BidAsk_price_gap_median,Bid_spread_median,Ask_spread_median,...,bid_size1_std,ask_size1_std,Wap1_std,Wap2_std,BidAsk_price_gap_std,Bid_spread_min,Ask_spread_std,trade_log_return1,about_price,seconds_in_bucket,price,size,order_count,max_price_range,max_price_size
0,0,0,4,0-4,0.000000,0.00000,0.0,0.0,0.000294,0.000252,1.000307,1.000475,0.000541,0.000393,0.000049,...,114.892704,46.188022,0.00017,0.000153,0.000028,0.0,0.000113,0.000295,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,4,0-4,1.000049,1.00059,91.0,100.0,0.000294,0.000252,1.000307,1.000475,0.000541,0.000393,0.000049,...,114.892704,46.188022,0.00017,0.000153,0.000028,0.0,0.000113,0.000295,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,4,0-4,1.000049,1.00059,91.0,100.0,0.000294,0.000252,1.000307,1.000475,0.000541,0.000393,0.000049,...,114.892704,46.188022,0.00017,0.000153,0.000028,0.0,0.000113,0.000295,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0,32,0-32,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0,34,0-34,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train=add_y(list_order_book_file_train,data_type='train').reset_index()
print(f'Train shape: {train.shape}')
# test.columns
# train.columns

0.89 %
1.79 %
2.68 %
3.57 %
4.46 %
5.36 %
6.25 %
7.14 %
8.04 %
8.93 %
9.82 %
10.71 %
11.61 %
12.5 %
13.39 %
14.29 %
15.18 %
16.07 %
16.96 %
17.86 %
18.75 %
19.64 %
20.54 %
21.43 %
22.32 %
23.21 %
24.11 %


### Model

In [ ]:
def train_and_evaluate(train, test):
    # Hyperparammeters (basic here, could be tuned with GridSearch later)
    params = {
      'objective': 'rmse',  
#         rmse regression
      'boosting_type': 'gbdt',
      'num_leaves': 100,
      'n_jobs': -1,
      'learning_rate': 0.2,
      'feature_fraction': 0.7,
      'bagging_fraction': 0.7,
      'max_depth': 4,
      'verbose': -1
    }
    
    # Split features and target
    x = train.drop(['row_id', 'target', 'time_id'], axis = 1)
    y = train['target']
    x_test = test.drop(['row_id','target', 'time_id'], axis = 1)
    
    # Transform stock id to a numeric value
    x['stock_id'] = x['stock_id'].astype(int)
    x_test['stock_id'] = x_test['stock_id'].astype(int)
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(x_test.shape[0])
    # Create a KFold object
    kfold = KFold(n_splits = 10, random_state = 66, shuffle = True)
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(x)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = x.iloc[trn_ind], x.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train, y_train, weight = train_weights, categorical_feature = ['stock_id'])
        val_dataset = lgb.Dataset(x_val, y_val, weight = val_weights, categorical_feature = ['stock_id'])
        model = lgb.train(params = params, 
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          num_boost_round = 10000, 
                          early_stopping_rounds = 100, 
                          verbose_eval = 100,
                          feval = feval_rmspe)
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val)
        # Predict the test set
        test_predictions += model.predict(x_test) / 5
        
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    # Return test predictions
    return test_predictions

In [ ]:
# Traing and evaluate
test_predictions = train_and_evaluate(train, test)
# Save test predictions

test['target'] = test_predictions
test[['row_id', 'target']].to_csv('submission.csv', index=False)